In [1]:
import pandas as pd

In [2]:
quote = pd.read_csv('trade_quote_data/quote.csv')
trade = pd.read_csv('trade_quote_data/trade.csv')

In [3]:
quote.head()

,date,sym,time,bid,ask,bsize,asize
0,2020-12-02,000001.SZSE,09:30:00.000,19.95,19.9600,127800.0,5800.0
1,2020-12-02,000001.SZSE,09:30:03.000,19.93,19.9399,6400.0,9282.0
2,2020-12-02,000001.SZSE,09:30:06.000,19.98,19.9899,39618.0,7200.0
3,2020-12-02,000001.SZSE,09:30:09.000,19.90,19.9300,43088.0,6200.0
4,2020-12-02,000001.SZSE,09:30:12.000,19.88,19.9000,116800.0,2026.0


In [4]:
trade['notional']=trade['price']*trade['size']
trade.head()

,date,sym,time,price,size,notional
0,2020-12-02,000001.SZSE,09:25:00.000,19.93,510912,10182476.16
1,2020-12-02,000001.SZSE,09:30:00.000,19.95,21600,430920.00
2,2020-12-02,000001.SZSE,09:30:03.000,19.93,568500,11330205.00
3,2020-12-02,000001.SZSE,09:30:06.000,19.98,106082,2119518.36
4,2020-12-02,000001.SZSE,09:30:09.000,19.91,70418,1402022.38


In [5]:
quote['minute']=quote['time'].apply(lambda x:x[:5])
trade['minute']=trade['time'].apply(lambda x:x[:5])

In [6]:
newtrade = trade.groupby(['date','sym']).agg({'price':'last','size':'sum','notional':'sum'})
newtrade.head()

price       size      notional
date       sym                                        
2020-12-02 000001.SZSE  19.63   88938529  1.758683e+09
           600030.SHSE  31.08  156773587  4.885569e+09
2020-12-03 000001.SZSE  19.54   71445230  1.393157e+09
           600030.SHSE  30.90   95761478  2.969124e+09
2020-12-04 000001.SZSE  19.30   89134784  1.707534e+09

In [7]:
n2trade = newtrade.groupby('sym')

In [8]:
tradedict ={}
for i,tradedf in n2trade:
    tradedf['return']=tradedf['price'].pct_change()
    tradedict[str(i)]=tradedf.copy()
    

C:\Users\Taichen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
tradedict['000001.SZSE'].groupby('sym').agg({'size':['mean'],'notional':['mean'],'return':['std']})

,size,notional,return
,mean,mean,std
sym,,,
000001.SZSE,8.342043e+07,1.563842e+09,0.016671


In [10]:
sz = tradedict['000001.SZSE'].groupby('sym').agg({'size':'mean','notional':'mean','return':'std'})
sh = tradedict['600030.SHSE'].groupby('sym').agg({'size':'mean','notional':'mean','return':'std'})

In [11]:
leftdf = pd.concat([sz,sh])
leftdf

,size,notional,return
sym,,,
000001.SZSE,8.342043e+07,1.563842e+09,0.016671
600030.SHSE,9.497093e+07,2.775817e+09,0.014878


In [54]:
# leftdf.columns.tolist()

['size', 'notional', 'return']

In [55]:
# leftdf=leftdf.reset_index()
# leftdf
# leftdf.columns=['Stock','ADV','ADTV','Volatility']

In [56]:
# leftdf.rename(columns={('sym', ''):'Stock', ('size', 'mean'):'ADV', ('notional', 'mean'):'ADVT', ('return', 'std'):'Volatility'},inplace=True)
# leftdf

,Stock,ADV,ADTV,Volatility
0,000001.SZSE,8.342043e+07,1.563842e+09,0.016671
1,600030.SHSE,9.497093e+07,2.775817e+09,0.014878


In [12]:
mintrade = trade.groupby(['date','sym','minute']).agg({'price':'last'})
mintrade_sym = mintrade.groupby(['date','sym'])

In [58]:
# mintrade.groupby(['date','sym','minute']).agg({'price':'pct_change'}).head(250)
# mintrade = mintrade.reindex()
# mintrade

In [13]:
mindict=[]
for syms,symtradedf in mintrade_sym:
    symtradedf['minret']=symtradedf['price'].pct_change()
    mindict.append(symtradedf.copy())

C:\Users\Taichen\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
pd.concat(mindict).groupby('sym').agg({'minret':'std'})

,minret
sym,
000001.SZSE,0.001528
600030.SHSE,0.001060


In [15]:
# middf = pd.concat([sz1,sh1])
middf = pd.concat(mindict).groupby('sym').agg({'minret':'std'})
t = 240**0.5
middf['Volatility5']=middf['minret']*t
middf

,minret,Volatility5
sym,,
000001.SZSE,0.001528,0.023668
600030.SHSE,0.001060,0.016415


In [16]:
pd.concat([leftdf,middf['Volatility5']],axis=1)

,size,notional,return,Volatility5
sym,,,,
000001.SZSE,8.342043e+07,1.563842e+09,0.016671,0.023668
600030.SHSE,9.497093e+07,2.775817e+09,0.014878,0.016415


In [17]:
quote["spread"] = 10000 * (quote["ask"] - quote["bid"]) / 0.5 / (quote["ask"] + quote["bid"])
quote["qsize"] = 0.5 * (quote["asize"] + quote["bsize"])
quote.head()

,date,sym,time,bid,ask,bsize,asize,minute,spread,qsize
0,2020-12-02,000001.SZSE,09:30:00.000,19.95,19.9600,127800.0,5800.0,09:30,5.011275,66800.0
1,2020-12-02,000001.SZSE,09:30:03.000,19.93,19.9399,6400.0,9282.0,09:30,4.966152,7841.0
2,2020-12-02,000001.SZSE,09:30:06.000,19.98,19.9899,39618.0,7200.0,09:30,4.953728,23409.0
3,2020-12-02,000001.SZSE,09:30:09.000,19.90,19.9300,43088.0,6200.0,09:30,15.064022,24644.0
4,2020-12-02,000001.SZSE,09:30:12.000,19.88,19.9000,116800.0,2026.0,09:30,10.055304,59413.0


In [18]:
rightdf = quote.groupby('sym').agg({'spread':'mean','qsize':'mean'})

In [19]:
q1 = pd.concat([leftdf,middf['Volatility5'],rightdf],axis=1)
q1

,size,notional,return,Volatility5,spread,qsize
sym,,,,,,
000001.SZSE,8.342043e+07,1.563842e+09,0.016671,0.023668,6.987517,34388.730735
600030.SHSE,9.497093e+07,2.775817e+09,0.014878,0.016415,4.128124,45190.873046


In [20]:
q1=q1.reset_index()
q1.columns=['Stock','ADV','ADTV','Volatility','Volatility5','Spread(bps)','Quote Size']
q1

,Stock,ADV,ADTV,Volatility,Volatility5,Spread(bps),Quote Size
0,000001.SZSE,8.342043e+07,1.563842e+09,0.016671,0.023668,6.987517,34388.730735
1,600030.SHSE,9.497093e+07,2.775817e+09,0.014878,0.016415,4.128124,45190.873046


In [21]:
def min5(a):
    minute = int(a[-2:])
    minutenew = int(minute/5)*5
    if len(str(minutenew)) == 1:
        minutenew = '0'+str(minutenew)
    b = a[:-2]+str(minutenew)
    return b

In [22]:
quote['mins5'] = quote['minute'].apply(min5)
trade['mins5'] = trade['minute'].apply(min5)
# c = pd.concat([new5,trade['minute']],axis=1
quote.head()

,date,sym,time,bid,ask,bsize,asize,minute,spread,qsize,mins5
0,2020-12-02,000001.SZSE,09:30:00.000,19.95,19.9600,127800.0,5800.0,09:30,5.011275,66800.0,09:30
1,2020-12-02,000001.SZSE,09:30:03.000,19.93,19.9399,6400.0,9282.0,09:30,4.966152,7841.0,09:30
2,2020-12-02,000001.SZSE,09:30:06.000,19.98,19.9899,39618.0,7200.0,09:30,4.953728,23409.0,09:30
3,2020-12-02,000001.SZSE,09:30:09.000,19.90,19.9300,43088.0,6200.0,09:30,15.064022,24644.0,09:30
4,2020-12-02,000001.SZSE,09:30:12.000,19.88,19.9000,116800.0,2026.0,09:30,10.055304,59413.0,09:30


In [23]:
quotenew = quote[quote['sym']=='600030.SHSE']
tradenew = trade[trade['sym']=='600030.SHSE']
quotenew.head()

,date,sym,time,bid,ask,bsize,asize,minute,spread,qsize,mins5
4762,2020-12-02,600030.SHSE,09:30:02.640,31.18,31.20,43200.0,2500.0,09:30,6.412312,22850.0,09:30
4763,2020-12-02,600030.SHSE,09:30:05.670,31.24,31.27,2800.0,3800.0,09:30,9.598464,3300.0,09:30
4764,2020-12-02,600030.SHSE,09:30:08.750,31.24,31.28,95800.0,72701.0,09:30,12.795905,84250.5,09:30
4765,2020-12-02,600030.SHSE,09:30:11.660,31.24,31.25,55200.0,200.0,09:30,3.200512,27700.0,09:30
4766,2020-12-02,600030.SHSE,09:30:14.680,31.17,31.20,1000.0,22100.0,09:30,9.620010,11550.0,09:30


In [24]:
# tradenew['ret']=tradenew['price'].pct_change()
tradenew.head()

,date,sym,time,price,size,notional,minute,mins5
4727,2020-12-02,600030.SHSE,09:25:00.030,31.30,1665800,52139540.0,09:25,09:25
4728,2020-12-02,600030.SHSE,09:30:02.640,31.25,527400,16481250.0,09:30,09:30
4729,2020-12-02,600030.SHSE,09:30:05.670,31.20,468800,14626560.0,09:30,09:30
4730,2020-12-02,600030.SHSE,09:30:08.750,31.24,177500,5545100.0,09:30,09:30
4731,2020-12-02,600030.SHSE,09:30:11.660,31.24,50900,1590116.0,09:30,09:30


In [25]:
tgdf= tradenew.groupby(['mins5']).agg({'size':'sum'})
total_size = tradenew['size'].sum()

In [26]:
volpct=tgdf/total_size

In [27]:
volpct.head()

,size
mins5,
09:25,0.005365
09:30,0.047318
09:35,0.036409
09:40,0.036977
09:45,0.032795


In [28]:
newret = pd.concat(mindict)

In [29]:
newret = newret.reset_index()

In [30]:
newret = newret[newret['sym']=='600030.SHSE']

In [31]:
newret['mins5'] = newret['minute'].apply(min5)
newret.head()

,date,sym,minute,price,minret,mins5
241,2020-12-02,600030.SHSE,09:25,31.30,NaN,09:25
242,2020-12-02,600030.SHSE,09:30,31.24,-0.001917,09:30
243,2020-12-02,600030.SHSE,09:31,31.35,0.003521,09:30
244,2020-12-02,600030.SHSE,09:32,31.33,-0.000638,09:30
245,2020-12-02,600030.SHSE,09:33,31.37,0.001277,09:30


In [32]:
newret[newret['mins5']=='09:30']

,date,sym,minute,price,minret,mins5
242,2020-12-02,600030.SHSE,09:30,31.24,-0.001917,09:30
243,2020-12-02,600030.SHSE,09:31,31.35,0.003521,09:30
244,2020-12-02,600030.SHSE,09:32,31.33,-0.000638,09:30
245,2020-12-02,600030.SHSE,09:33,31.37,0.001277,09:30
246,2020-12-02,600030.SHSE,09:34,31.36,-0.000319,09:30
722,2020-12-03,600030.SHSE,09:30,30.96,-0.002256,09:30
723,2020-12-03,600030.SHSE,09:31,31.04,0.002584,09:30
724,2020-12-03,600030.SHSE,09:32,30.96,-0.002577,09:30
725,2020-12-03,600030.SHSE,09:33,30.99,0.000969,09:30
726,2020-12-03,600030.SHSE,09:34,30.94,-0.001613,09:30


In [33]:
0.02/29.04

0.0006887052341597796

In [34]:
minretstd = newret.groupby('mins5').agg({'minret':'std'})
minretstd

,minret
mins5,
09:25,NaN
09:30,0.001761
09:35,0.001784
09:40,0.001483
09:45,0.001351
09:50,0.001363
09:55,0.001365
10:00,0.001243
10:05,0.001081


In [35]:
qsmean = quotenew.groupby('mins5').agg({'spread':'mean','qsize':'mean'})
qsmean

,spread,qsize
mins5,,
09:30,5.088313,27938.186876
09:35,4.362966,27879.402001
09:40,4.435114,31338.251704
09:45,4.361489,27779.665757
09:50,4.295296,34625.167273
09:55,4.443551,31309.641008
10:00,4.314098,30696.606639
10:05,4.261098,30994.018190
10:10,4.165040,32976.812982


In [36]:
q2 = pd.concat([minretstd,qsmean,volpct],axis=1)
q2.head()

C:\Users\Taichen\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,minret,spread,qsize,size
09:25,NaN,NaN,NaN,0.005365
09:30,0.001761,5.088313,27938.186876,0.047318
09:35,0.001784,4.362966,27879.402001,0.036409
09:40,0.001483,4.435114,31338.251704,0.036977
09:45,0.001351,4.361489,27779.665757,0.032795


In [122]:
cai = pd.read_csv('asgn1_2.csv')

In [123]:
cai.head()

,minute,vol5,Spread,QSize,volpct
0,09:25,NaN,NaN,NaN,0.005085
1,09:30,0.027156,5.088313,27938.19,0.045619
2,09:35,0.027507,4.362966,27879.40,0.035594
3,09:40,0.022874,4.435114,31338.25,0.036428
4,09:45,0.020840,4.361489,27779.67,0.031693


In [124]:
cai['volpct'].sum()

0.999999971

In [125]:
cai1 = pd.read_csv('asgn1_1.csv')

In [126]:
cai1

,sym,ADV,ADTV,Volatility,Volatility5,Spread,QSize
0,000001.SZSE,83420430.0,1.563842e+09,0.016270,0.022272,6.987517,34388.73
1,600030.SHSE,94970930.0,2.775817e+09,0.014519,0.015740,4.128124,45190.87


In [127]:
q1

,Stock,ADV,ADTV,Volatility,Volatility5,Spread(bps),Quote Size
0,000001.SZSE,8.342043e+07,1.563842e+09,0.016671,0.023668,6.987517,34388.730735
1,600030.SHSE,9.497093e+07,2.775817e+09,0.014878,0.016415,4.128124,45190.873046


In [128]:
volpct.sum()

size    1.0
dtype: float64

In [131]:
trade.describe()

,price,size,notional
count,206357.000000,2.063570e+05,2.063570e+05
mean,23.916752,1.901855e+04,4.626570e+05
std,5.231670,4.323692e+04,1.119899e+06
min,17.790000,2.700000e+01,4.860000e+02
25%,18.759900,3.400000e+03,7.818200e+04
50%,27.740000,8.400000e+03,1.968750e+05
75%,28.850000,1.980000e+04,4.727922e+05
max,31.600000,3.877067e+06,1.104964e+08


In [132]:
quote.describe()

,bid,ask,bsize,asize,spread,qsize
count,210347.000000,210347.000000,2.103470e+05,2.103470e+05,210347.000000,2.103470e+05
mean,23.926329,23.938875,4.280882e+04,3.681715e+04,5.551683,3.981299e+04
std,5.230796,5.230291,1.186552e+05,1.045336e+05,3.397952,1.020692e+05
min,17.790000,17.800000,1.000000e+00,1.000000e+00,0.000000,5.200000e+01
25%,18.750000,18.770000,3.600000e+03,3.848000e+03,3.475239,7.950000e+03
50%,27.790000,27.800000,1.360000e+04,1.320000e+04,5.218303,1.885000e+04
75%,28.850000,28.860000,4.010000e+04,3.620000e+04,5.492996,4.112975e+04
max,31.590000,31.600000,3.799167e+06,3.799167e+06,58.839262,3.799167e+06


In [37]:
q2 = q2.reset_index()

In [38]:
q2.columns = ['time','vol5','spread','qsize','volpct']

In [39]:
q2.head()

,time,vol5,spread,qsize,volpct
0,09:25,NaN,NaN,NaN,0.005365
1,09:30,0.001761,5.088313,27938.186876,0.047318
2,09:35,0.001784,4.362966,27879.402001,0.036409
3,09:40,0.001483,4.435114,31338.251704,0.036977
4,09:45,0.001351,4.361489,27779.665757,0.032795


In [145]:
q1

,Stock,ADV,ADTV,Volatility,Volatility5,Spread(bps),Quote Size
0,000001.SZSE,8.342043e+07,1.563842e+09,0.016671,0.023668,6.987517,34388.730735
1,600030.SHSE,9.497093e+07,2.775817e+09,0.014878,0.016415,4.128124,45190.873046


In [40]:
# q1.to_excel('asgn1_1.xlsx',index=False)
# q2.to_excel('asgn1_2.xlsx',index=False)